In [1]:
from fastai.vision.all import *
from fastai.learner import *
from fastai.data.all import *
from fastai.callback.tracker import SaveModelCallback
import pandas as pd
import matplotlib.pyplot as plt
from pathlib2 import Path
import numpy as np
import random
from torch.nn import MSELoss

In [2]:
%%time

df = pd.read_csv('data/examples_with_length_speech2vec_vocab.csv')
df.shape

CPU times: user 27.1 s, sys: 3.06 s, total: 30.2 s
Wall time: 45.5 s


(14276908, 12)

In [3]:
%%time
fn2features = pd.read_pickle('data/fn2feature.pkl')

CPU times: user 17.7 s, sys: 13.2 s, total: 30.9 s
Wall time: 1min 34s


In [4]:
# def prepare_features(fn, pad_to=70, pad_left=False):
#     ary = fn2features[fn][:pad_to]
#     example = np.zeros((pad_to, 13))
#     if pad_left:
#         example[-ary.shape[0]:, :] = ary
#     else: example[:ary.shape[0], :] = ary
#     return example.astype(np.float32)

In [5]:
# dataset_mean = -2
# dataset_std = 10

# def normalize_data(ary):
#     return (ary - dataset_mean) / dataset_std

In [6]:
# %%time

# features = []
# for fn in df.target_fn.unique():
#     features.append(normalize_data(prepare_features(fn)))

In [7]:
# np.stack(features).mean(), np.stack(features).std()

In [8]:
# fn2features_norm = {fn: normalize_data(prepare_features(fn)) for fn, features in fn2features.items()}
# pd.to_pickle(fn2features_norm, 'data/fn2features_norm.pkl')

In [4]:
%%time
fn2features = pd.read_pickle('data/fn2features_norm.pkl')

CPU times: user 16.2 s, sys: 10.1 s, total: 26.3 s
Wall time: 26.3 s


In [5]:
df = df[df.distance_from_target == 1]
df = df[(~df.target_word.isna() & ~df.source_word.isna())]
df = df[df.in_speech2vec_vocab]
df = df[((df.source_length < 71) & (df.target_length < 71) & (df.source_length > 19))]
df.reset_index(inplace=True, drop=True)

In [6]:
words = np.concatenate((df.target_word, df.source_word))
vocab = list(np.unique(words))

In [7]:
def empty_list(): return list()

In [8]:
# %%time

# word2row_idxs = defaultdict(empty_list)

# for idx, row in df.iterrows():
#     word2row_idxs[row.source_word].append(idx)    

# pd.to_pickle(word2row_idxs, 'data/word2row_idxs_speech2vec_vocab_min_length_20_max_length_70_only_driect_neighbors.pkl')

CPU times: user 6min, sys: 488 ms, total: 6min
Wall time: 6min


In [9]:
word2row_idxs = pd.read_pickle('data/word2row_idxs_speech2vec_vocab_min_length_20_max_length_70_only_driect_neighbors.pkl')
word2index = {word: i for i, word in enumerate(vocab)}

In [10]:
train_examples = df[df.set_name.isin(['train-clean-360', 'train-clean-100', 'dev-clean'])]
valid_examples = df[df.set_name == 'test-clean']

train_examples.reset_index(inplace=True, drop=True)
valid_examples.reset_index(inplace=True, drop=True)

In [11]:
class Dataset():
    def __init__(self, df):
        self.df = df
    def __len__(self):
        return self.df.shape[0]
    def __getitem__(self, idx):
        source_fn = self.df.source_fn[idx]
        target_word = self.df.target_word[idx]
        return fn2features[source_fn], word2index[target_word]

In [12]:
train_ds = Dataset(train_examples)
valid_ds = Dataset(valid_examples)

len(train_ds), len(valid_ds)

(4131148, 46189)

In [13]:
BS = 2048
NUM_WORKERS = 8

train_dl = DataLoader(train_ds, BS, NUM_WORKERS, shuffle=True)
valid_dl = DataLoader(valid_ds, BS, NUM_WORKERS)

dls = DataLoaders(train_dl, valid_dl)

In [14]:
# Got the following error while training:

# DataLoader worker (pid 2073) is killed by signal: Bus error. It is possible that dataloader's workers are out of shared memory. Please try to raise your shared memory limit.
# trying the solution I found here: https://github.com/pytorch/pytorch/issues/5040
# which is to execute
!sudo umount /dev/shm/ && sudo mount -t tmpfs -o rw,nosuid,nodev,noexec,relatime,size=50G shm /dev/shm

In [59]:
class Model(Module):
    def __init__(self, hidden_size=150):
        self.return_embeddings = False
        self.hidden_size = hidden_size
        
        self.encoder= nn.LSTM(
            input_size=13,
            hidden_size=hidden_size,
            num_layers=3,
            batch_first=True,
            dropout=0,
            bidirectional=True
        )
        
        self.linear = nn.Linear(2*hidden_size, len(vocab))
            
    def forward(self, source_features):
        _, (embeddings, _) = self.encoder(source_features)      
        
        # embeddings.view(num_layers, num_directions, batch, hidden_size)
        embeddings = embeddings.view(3, 2, source_features.shape[0], self.hidden_size)
        embeddings = torch.cat((embeddings[-1, 0], embeddings[-1, 1]), -1)
        
        if self.return_embeddings: return embeddings

        return self.linear(embeddings)

In [83]:
learn = Learner(
    dls.cuda(),
    Model().cuda(),
    loss_func=CrossEntropyLossFlat(),
    opt_func=Adam,
    metrics=[accuracy]
)

In [ ]:
NUM_EPOCHS = 144
learn.fit(NUM_EPOCHS, lr=1e-3, cbs=SaveModelCallback(fname='rnn_encoder', every_epoch=True))

epoch,train_loss,valid_loss,None,time


## Calculate embedding for each unique word in the dataset

In [ ]:
df_unique_utterances = df[df.set_name.isin(['train-clean-360', 'train-clean-100', 'dev-clean'])].drop_duplicates(['source_fn'])
df_unique_utterances.reset_index(drop=True, inplace=True)

all_dl = DataLoader(Dataset(df_unique_utterances), BS, NUM_WORKERS)

In [ ]:
%%time

learn.model.return_embeddings = True
learn.model.eval()

all_embeddings = []

with torch.no_grad():    
    for batch in all_dl:
        embeddings = learn.model(batch[0].cuda())
        all_embeddings.append(embeddings.detach().cpu().squeeze(0))

In [ ]:
all_embeddings = torch.cat(all_embeddings)
all_embeddings.shape

In [ ]:
%%time

word2row_idxs_unique_utterances = defaultdict(empty_list)

for idx, row in df_unique_utterances.iterrows():
    word2row_idxs_unique_utterances[row.source_word].append(idx)
    
pd.to_pickle(word2row_idxs_unique_utterances, 'word2row_idxs_unique_utterances_speech2vec_vocab_min_length_20_max_length_70_only_driect_neighbors.pkl')

In [ ]:
word2row_idxs_unique_utterances = pd.read_pickle('word2row_idxs_unique_utterances_speech2vec_vocab_min_length_20_max_length_70_only_driect_neighbors.pkl')

In [ ]:
word2embedding = {}

for k, v in word2row_idxs_unique_utterances.items():
    word2embedding[k] = all_embeddings[np.array(v)].mean(0).numpy()

In [ ]:
from utils import Embeddings

In [ ]:
e = Embeddings(
    np.array(list(word2embedding.values())),
    [w.lower() for w in list(word2embedding.keys())]
)

In [ ]:
for w in ['fast', 'lost', 'small', 'true', 'crazy', 'slow']:
    print(f'{w}: {e.nn_words_to(e[w])}')

## Evaluating embeddings using [word-embeddings-benchmarks](https://github.com/kudkudak/word-embeddings-benchmarks)

In [ ]:
import logging
from six import iteritems
from web.datasets.similarity import fetch_MEN, fetch_WS353, fetch_SimLex999
from web.embeddings import fetch_GloVe
from web.evaluate import evaluate_similarity
from web.embedding import Embedding, Vocabulary
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [ ]:
tasks = {
    "MEN": fetch_MEN(),
    "WS353": fetch_WS353(),
    "SIMLEX999": fetch_SimLex999()
}

In [ ]:
our_embeddings = Embedding(
    Vocabulary([w.lower() for w in list(word2embedding.keys())]),
    np.array(list(word2embedding.values()))
)

speech2vec = KeyedVectors.load_word2vec_format('../speech2vec-pretrained-vectors/speech2vec/50.vec', binary=False) 
speech2vec_embeddings = Embedding(Vocabulary(list(speech2vec.vocab.keys())), speech2vec.vectors)

In [ ]:
for name, data in iteritems(tasks):
    print("Spearman correlation of scores on {} {}".format(name, evaluate_similarity(our_embeddings, data.X, data.y)))

In [ ]:
for name, data in iteritems(tasks):
    print("Spearman correlation of scores on {} {}".format(name, evaluate_similarity(speech2vec_embeddings, data.X, data.y)))

## Loss decrease and improvements on semantic tasks as training progresses

In [ ]:
%%capture

val_losses, accuracies, task_perf = [], [], []
for i in range(NUM_EPOCHS):
    learn.load(f'rnn_encoder_{i}')
    learn.model.return_embeddings = False
    loss, accuracy = learn.validate()
    val_losses.append(loss)
    accuracies.append(accuracy)
    
    learn.model.return_embeddings = True
    learn.model.eval()

    all_embeddings = []

    with torch.no_grad():    
        for batch in all_dl:
            embeddings = learn.model(batch[0].cuda())
            all_embeddings.append(embeddings.detach().cpu().squeeze(0))

    all_embeddings = torch.cat(all_embeddings)

    word2embedding = {}
    for k, v in word2row_idxs_unique_utterances.items():
        word2embedding[k] = all_embeddings[np.array(v)].mean(0).numpy()

    our_embeddings = Embedding(
        Vocabulary([w.lower() for w in list(word2embedding.keys())]),
        np.array(list(word2embedding.values()))
    )
    task_perf.append([evaluate_similarity(our_embeddings, data.X, data.y) for name, data in iteritems(tasks)])

In [ ]:
men, ws353, simlex999 = list(zip(*task_perf))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax1 = plt.subplots()

ax1.plot(val_losses, label='val loss')

ax2 = ax1.twinx()
ax2.plot(men, label='MEN', c='g')
ax2.plot(ws353, label='WS353', c='m')
ax2.plot(simlex999, label='SIMLEX999', c='y')

ax1.legend(loc=[0.07, 0.9])
ax2.legend(loc=[0.7, 0.15])

ax1.set_xlabel('epochs');